In [ ]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
from uniswap import Uniswap
import time
from dotenv import load_dotenv
import os
import json
import requests
import numpy as np

In [ ]:
class TokenTrader:
    def __init__(self, rpc_url, wallet_address, private_key, token_address, 
                 usdt_address, eth_address, price_thresholds, lot_size_tokens, 
                 min_sell_amount_tokens, etherscan_api_key, contract_address, presale_id):
        self.rpc_url = rpc_url
        self.wallet_address = wallet_address
        self.private_key = private_key
        self.token_address = token_address
        self.usdt_address = usdt_address
        self.eth_address = eth_address
        self.price_thresholds = price_thresholds
        self.lot_size_tokens = lot_size_tokens
        self.min_sell_amount_tokens = min_sell_amount_tokens
        self.etherscan_api_key = etherscan_api_key
        self.contract_address = contract_address
        self.presale_id = presale_id
        
        self.web3 = Web3(Web3.HTTPProvider(self.rpc_url))
        self.web3.eth.set_gas_price_strategy(rpc_gas_price_strategy)
        self.uniswap = Uniswap(address=self.wallet_address, private_key=self.private_key, version=2, web3=self.web3, provider=self.rpc_url)
        
        self.contract = self.web3.eth.contract(address=Web3.to_checksum_address(self.contract_address), abi=self.get_contract_abi())
        self.eth_decimals = self.get_token_decimals(self.eth_address)
        self.usdt_decimals =self.get_token_decimals(self.usdt_address)
        self.token_decimals = self.get_token_decimals(self.token_address)
        self.lot_size = int(self.lot_size_tokens * 10**self.token_decimals)
        self.min_sell_amount = int(self.min_sell_amount_tokens * 10**self.token_decimals)

        self.state_file = 'trader_state.txt'
        self.sold_lots, self.sold_tokens = self.load_state()

    def load_state(self):
        if os.path.exists(self.state_file):
            with open(self.state_file, 'r') as file:
                state = json.load(file)
                sold_lots = int(state['sold_lots'])
                sold_tokens = [int(token) for token in state['sold_tokens']]
        else:
            sold_lots = 0
            sold_tokens = [0] * len(self.price_thresholds)
        return sold_lots, sold_tokens

    def save_state(self):
        state = {
            'sold_lots': str(self.sold_lots),
            'sold_tokens': [str(token) for token in self.sold_tokens] 
        }
        with open(self.state_file, 'w') as file:
            json.dump(state, file)
    
    def get_contract_abi(self):
        try:
            url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={self.contract_address}&apikey={self.etherscan_api_key}"
            response = requests.get(url)
            response_json = response.json()
            # Comprueba si la solicitud fue exitosa y si hay un ABI disponible
            if response_json['status'] == '1' and response_json['message'] == 'OK':
                abi = json.loads(response_json['result'])
                return abi
        except Exception as e:
            print(f"Ha ocurrido un error al obtener el ABI: {e}")
            return None
    
    def claim_tokens(self):
        claim_txn = self.contract.functions.claimAmount(self.presale_id).buildTransaction({
            'from': self.wallet_address,
            'nonce': self.web3.eth.get_transaction_count(self.wallet_address),
            'gas': 2000000
        })
        signed_txn = self.web3.eth.account.sign_transaction(claim_txn, private_key=self.private_key)
        tx_hash = self.web3.eth.send_raw_transaction(signed_txn.rawTransaction)
        receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Tokens reclamados con éxito, transacción: {receipt.transactionHash.hex()}")
        return receipt

    def attempt_to_claim_tokens(self):
        try:
            # Verificar si el reclamo está habilitado para esta presale
            presale_info = self.contract.functions.presale(self.presale_id).call()
            if not presale_info[9]:  # isEnableClaim está en la posición 9 según el ABI
                print("El reclamo para esta presale aún no está habilitado.")
                return
            # Consultar la cantidad reclamable
            claimable_amount = self.contract.functions.claimableAmount(self.wallet_address, self.presale_id).call()
            if claimable_amount == 0:
                print("No hay tokens disponibles para reclamar en este momento.")
                return
            # Si el reclamo está habilitado y hay tokens disponibles, proceder a reclamar
            receipt = self.claim_tokens(self.presale_id)
            if receipt:
                print(f"Reclamo realizado con éxito. Transacción: {receipt.transactionHash.hex()}")
            else:
                print("Reclamo fallido.")
        except Exception as e:
            print(f"Error al intentar reclamar tokens: {e}")

    def get_token_decimals(self, token_address):
        token_contract = self.uniswap.get_token(token_address)
        return token_contract.decimals

    def get_token_balance(self, token_address):
        return self.uniswap.get_token_balance(token_address)
    
    def get_price_from_pool(self, input_token, output_token, decimals):
        try:
            price = self.uniswap.get_price_input(input_token, output_token, 10**decimals)
            return price
        except Exception as e:
            print(f"Ha ocurrido un error al chequear el precio: {e}")
            return None

    def get_token_price_in_usdt(self):
        token_price_in_eth = self.get_price_from_pool(self.token_address, self.eth_address, self.token_decimals)
        eth_price_in_usdt = self.get_price_from_pool(self.eth_address, self.usdt_address, self.eth_decimals)
        if token_price_in_eth is not None and eth_price_in_usdt is not None:
            return (token_price_in_eth / 10**self.eth_decimals) * (eth_price_in_usdt / 10**self.usdt_decimals)
        return None

    def sell_token_for_base(self, token_address, base_address, amount):
        tx = self.uniswap.make_trade(token_address, base_address, int(amount))
        txid = tx.hex()
        print(f"Esperando por confirmación de la transacción {txid}")
        self.web3.eth.wait_for_transaction_receipt(tx)
        return txid
    
    def manage_sales(self, balance, price_in_usdt):
         # Actualiza el lote actual basado en el precio
        while self.sold_lots+1 < len(self.price_thresholds) and price_in_usdt >= self.price_thresholds[self.sold_lots+1]:
            self.sold_lots += 1
        # Determinar cantidad total a vender basada en los lotes pendientes
        total_to_sell = 0
        for i in range(self.sold_lots+1):
            total_to_sell += self.lot_size - self.sold_tokens[i]
        amount_to_sell = min(balance, total_to_sell)
        # Vender si hay algo que vender y el precio es el adecuado
        if amount_to_sell >= self.min_sell_amount and price_in_usdt >= self.price_thresholds[self.sold_lots]:
            print(f"Vendiendo {(amount_to_sell / 10**self.token_decimals)} tokens a {price_in_usdt} USDT")
            tx_receipt = self.sell_token_for_base(self.token_address, self.eth_address, amount_to_sell)
            print(f"Transacción confirmada:")# {tx_receipt}")
            # Actualizar estado según lo vendido
            sold = amount_to_sell
            for i in range(self.sold_lots + 1):
                if sold <= 0:
                    break
                sell_from_lot = min(sold, self.lot_size - self.sold_tokens[i])
                self.sold_tokens[i] += sell_from_lot
                sold -= sell_from_lot
            # Guardar estado
            self.save_state()
            # Pasar al siguiente lote
            if self.sold_tokens[self.sold_lots] >= self.lot_size:
                self.sold_lots += 1

    def trade(self):
        while self.sold_lots < len(self.price_thresholds):
            try:
                self.attempt_to_claim_tokens()
                balance = int(self.get_token_balance(self.token_address))
                print(f"Balance actual: {(balance / 10**self.token_decimals)} tokens")
                price_in_usdt = self.get_token_price_in_usdt()
                print(f"Precio actual: {price_in_usdt} USDT")
                if price_in_usdt is not None and balance > self.min_sell_amount:
                    self.manage_sales(balance, price_in_usdt)
                else:
                    time.sleep(5)
                    continue
                # Espero 1 segundo
                time.sleep(1)
            except Exception as e:
                print(f"Ha ocurrido un error: {e}")
                time.sleep(1)

In [ ]:
if __name__ == "__main__":
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    rpc_url = "https://rpc.ankr.com/eth" 
    wallet_address = os.getenv("WALLET_ADDRESS")
    private_key = os.getenv("PRIVATE_KEY")
    etherscan_api_key = os.getenv("ETHERSCAN_API_KEY")
    eth_address = "0x0000000000000000000000000000000000000000"
    usdt_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
    token_address = "0x26EbB8213fb8D66156F1Af8908d43f7e3e367C1d"
    token_contract = "0x602C90D796D746b97a36f075d9f3b2892B9B07c2"
    presale_id = 2
    price_thresholds = np.linspace(0.49, 1.99, 20)
    lot_size_tokens = 2547
    min_sell_amount_tokens = 1000

    trader = TokenTrader(rpc_url, wallet_address, private_key, token_address, usdt_address, eth_address, price_thresholds,
                         lot_size_tokens, min_sell_amount_tokens, etherscan_api_key, token_contract, presale_id)
    trader.trade()